In [1]:
import Bio
import numpy as np
from Bio import SeqIO
import sys
import csv
import os
import random as random
from functions_collection_copy import *

score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5

In [2]:
def make_related_strings(nr_of_str:int,len_of_str, degree_of_variation:float, start_string):
    string_fam_collection=[]
    first_string=[]
    if start_string==False:
        for k in range(0,int(len_of_str)):
                first_string.append(random.choice(['a','c','t','g']))
        first_string=''.join(first_string)
        string_fam_collection.append(first_string)
    else:
         first_string=start_string
         string_fam_collection.append(first_string)

    for i in range(1,nr_of_str):
        a_sequence=[]
        for element in first_string:
            if random.random() < degree_of_variation: #checking if we should change the charachter
                a_sequence.append(random.choice(['a','c','t','g']))
            else:
                 a_sequence.append(element)
        a_sequence=''.join(a_sequence)
        string_fam_collection.append(a_sequence)
    return string_fam_collection

#make_related_strings(3,12,0.1,False)


def make_strings_in_families(nr_of_fams, nr_str_pr_fam, len_of_str, internal_var_in_fams, degree_of_var_from_first_fam):
    all_strings_coll = []
    first_family = make_related_strings(nr_str_pr_fam, len_of_str, internal_var_in_fams, start_string=False)
    all_strings_coll.append(first_family)
    
    for i in range(1, int(nr_of_fams)):
        carry_over_string = first_family[0]
        mutated_carry_over = []
        for element in carry_over_string:
            if random.random() < degree_of_var_from_first_fam:  # checking if we should change the character
                mutated_carry_over.append(random.choice(['a', 'c', 't', 'g']))
            else:
                mutated_carry_over.append(element)
        mutated_carry_over = ''.join(mutated_carry_over)  # Join the list of characters into a string
        family = make_related_strings(nr_str_pr_fam, len_of_str, internal_var_in_fams,mutated_carry_over)
        all_strings_coll.append(family)
    all_strings_coll = [item for sublist in all_strings_coll for item in sublist]
    #all_strings_coll = [''.join(sublist) for sublist in all_strings_coll]
    return all_strings_coll


In [3]:
def test_many_parameters_no_internal_reps(nr_of_str=10,len_str=20,nr_of_fams=2, str_pr_fam=5, internal_var=0.1, var_between_fams=0.9,mode='random' ):
    costs_Krus=float('inf')
    costs_Prim=float('inf')
    costs_Gus=float('inf')
    if mode=='random':
        random_sequences_collection=[]
        random_pile=[]
        for j in range (0,nr_of_str):
            a_sequence=[]
            for k in range(0,len_str):
                a_sequence.append(random.choice(['a','c','t','g']))
            a_sequence=''.join(a_sequence)
            random_pile.append(a_sequence)
        random_sequences_collection.append(random_pile)
        for random_pile in random_sequences_collection:
            #print('rand: I intend to align: '+str(random_pile))
            cost=new_assembly_gradual_x(random_pile,score_matrix,gap_cost)[2]
            costs_Krus=cost
        for random_pile in random_sequences_collection:
            cost=new_assembly_Prim_x(random_pile,score_matrix,gap_cost)[2]
            costs_Prim=cost
        for random_pile in random_sequences_collection:
            #print('rand:I intend to align: '+str(random_pile))
            cost=new_assembly_Gus_x(random_pile,score_matrix,gap_cost)[2]
            costs_Gus=cost
    elif mode=='family':
        #internal_var=[internal_var]*nr_of_fams
        #var_between_fams=[var_between_fams]*nr_of_fams
        strings_in_families=make_strings_in_families(nr_of_fams,str_pr_fam,len_str,internal_var,var_between_fams)
        #print('fam:I intend to align: '+str(strings_in_families))
        cost=new_assembly_gradual_x(strings_in_families,score_matrix,gap_cost)[2]

        costs_Krus=cost

        cost=new_assembly_Prim_x(strings_in_families,score_matrix,gap_cost)[2]
        costs_Prim=cost
        
        cost=new_assembly_Gus_x(strings_in_families,score_matrix,gap_cost)[2]
        costs_Gus=cost
        
    else:
        print("that's not a valid mode, try 'family' or 'random'!")
    #result=costs_new_gradual, costs_new_one_by_one, costs_old
    #result=costs_Krus,costs_Prim,costs_Gus
    if mode=='random':
        #params="nr str: "+ str(nr_of_str)+", str len: "+str(len_str)+",mode:"+str(mode)
        #params=nr_of_str,len_str,mode
        return(nr_of_str,len_str,mode,costs_Krus,costs_Prim,costs_Gus)
    if mode=='family':
        #params="str len: "+str(len_str)+", nr fams: "+str(nr_of_fams)+", str_pr_fam: "+str(str_pr_fam)+", internal var: "+ str(internal_var)+", var between fams: "+str(var_between_fams)+", mode:"+str(mode)
        return (len_str,nr_of_fams,str_pr_fam,internal_var,var_between_fams,mode,costs_Krus,costs_Prim,costs_Gus)

test_many_parameters_no_internal_reps(len_str=15,nr_of_fams=3,str_pr_fam=3,internal_var=0.1,var_between_fams=0.9, mode='random')


    
        

(10, 15, 'random', 2728, 2728, 2162)

In [8]:
def test_many_parameters_no_internal_reps_new_prim(nr_of_str=10,len_str=20,nr_of_fams=2, str_pr_fam=5, internal_var=0.1, var_between_fams=0.9,mode='random' ):
    costs_Krus=float('inf')
    costs_Prim=float('inf')
    costs_Gus=float('inf')
    if mode=='random':
        random_sequences_collection=[]
        random_pile=[]
        for j in range (0,nr_of_str):
            a_sequence=[]
            for k in range(0,len_str):
                a_sequence.append(random.choice(['a','c','t','g']))
            a_sequence=''.join(a_sequence)
            random_pile.append(a_sequence)
        random_sequences_collection.append(random_pile)
        for random_pile in random_sequences_collection:
            #print('rand: I intend to align: '+str(random_pile))
            cost=new_assembly_gradual_x(random_pile,score_matrix,gap_cost)[2]
            costs_Krus=cost
        for random_pile in random_sequences_collection:
            cost=new_assembly_Primzeyy_x(random_pile,score_matrix,gap_cost,check_integrity=True)[2]
            costs_Prim=cost
        for random_pile in random_sequences_collection:
            #print('rand:I intend to align: '+str(random_pile))
            cost=new_assembly_Gus_x(random_pile,score_matrix,gap_cost)[2]
            costs_Gus=cost
    elif mode=='family':
        #internal_var=[internal_var]*nr_of_fams
        #var_between_fams=[var_between_fams]*nr_of_fams
        strings_in_families=make_strings_in_families(nr_of_fams,str_pr_fam,len_str,internal_var,var_between_fams)
        #print('fam:I intend to align: '+str(strings_in_families))
        cost=new_assembly_gradual_x(strings_in_families,score_matrix,gap_cost)[2]

        costs_Krus=cost

        cost=new_assembly_Primzeyy_x(strings_in_families,score_matrix,gap_cost,check_integrity=True)[2]
        costs_Prim=cost
        
        cost=new_assembly_Gus_x(strings_in_families,score_matrix,gap_cost)[2]
        costs_Gus=cost
        
    else:
        print("that's not a valid mode, try 'family' or 'random'!")
    #result=costs_new_gradual, costs_new_one_by_one, costs_old
    #result=costs_Krus,costs_Prim,costs_Gus
    if mode=='random':
        #params="nr str: "+ str(nr_of_str)+", str len: "+str(len_str)+",mode:"+str(mode)
        #params=nr_of_str,len_str,mode
        return(nr_of_str,len_str,mode,costs_Krus,costs_Prim,costs_Gus)
    if mode=='family':
        #params="str len: "+str(len_str)+", nr fams: "+str(nr_of_fams)+", str_pr_fam: "+str(str_pr_fam)+", internal var: "+ str(internal_var)+", var between fams: "+str(var_between_fams)+", mode:"+str(mode)
        return (len_str,nr_of_fams,str_pr_fam,internal_var,var_between_fams,mode,costs_Krus,costs_Prim,costs_Gus)

test_many_parameters_no_internal_reps(len_str=15,nr_of_fams=3,str_pr_fam=3,internal_var=0.1,var_between_fams=0.9, mode='random')


    
        

(10, 15, 'random', 2443, 2443, 2263)

In [9]:
#for random costs
repeats=3
mode='random'
str_len=[10,50,100,200,300,500,750,1000]
nr_str=[5,10,15]

csv_file_path="testing_outputs/random_strs_tests_new_prim.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile, delimiter=',', quotechar=' ', quoting=csv.QUOTE_NONE)
                            csvwriter.writerow(["nr_of_str","str_len","mode","Kruskal","Prim","Gusfield"])

for i in range(0,repeats):
    for lens in str_len:
        for nr in nr_str:

            nr_of_str,len_str,mode,costs_Krus,costs_Prim,costs_Gus=test_many_parameters_no_internal_reps(nr,lens,mode='random')
            with open(csv_file_path, 'a', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow([nr_of_str,len_str,mode,costs_Krus,costs_Prim,costs_Gus])

KeyboardInterrupt: 

In [150]:
#for families cost
repeats=3
mode='family'
nr_fams=[1,2,4]
str_pr_fam=[3,10]
internal_var=[0.1,0.9]
var_between_fams=[0.1,0.9]
str_len=[10,50,100,200,400]
for x in var_between_fams:
    print(x)
csv_file_path="testing_outputs/familiy_outputs_testing2.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow(["str_len","nr_of_fams","str_pr_fam","internal_var","external_var","mode","Kruskal","Prim","Gusfield"])

for i in range(0,repeats):
    for fam_num in nr_fams:
        for str_pr_fam_nr in str_pr_fam:
            for int_var in internal_var:
                for ex_var in var_between_fams:
                    for str_l in str_len:

                        len_str,nr_of_fams,strs_pr_fam,inte_var,var_between_fam,mode,costs_Krus,costs_Prim,costs_Gus=test_many_parameters_no_internal_reps(len_str=str_l,nr_of_fams=fam_num,internal_var=int_var,str_pr_fam=str_pr_fam_nr,var_between_fams=ex_var,mode=mode)

                        with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow([len_str,nr_of_fams,strs_pr_fam,inte_var,var_between_fam,mode,costs_Krus,costs_Prim,costs_Gus])

0.1
0.9


































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































KeyboardInterrupt: 

In [4]:
import time
import random


def test_many_parameters_no_internal_reps_time(nr_of_str=10, len_str=20, nr_of_fams=2, str_pr_fam=5, internal_var=0.1, var_between_fams=0.9, mode='random'):
    time_taken_Krus = float('inf')
    time_taken_Prim = float('inf')
    time_taken_Gus = float('inf')

    start_time = time.time()

    if mode == 'random':
        random_sequences_collection = []
        random_pile = []
        for j in range(0, nr_of_str):
            a_sequence = []
            for k in range(0, len_str):
                a_sequence.append(random.choice(['a', 'c', 't', 'g']))
            a_sequence = ''.join(a_sequence)
            random_pile.append(a_sequence)
        random_sequences_collection.append(random_pile)
        for random_pile in random_sequences_collection:
            start_func_time = time.time()
            new_assembly_gradual_x(random_pile, score_matrix, gap_cost)
            end_func_time = time.time()
            time_taken_Krus = end_func_time - start_func_time

        for random_pile in random_sequences_collection:
            start_func_time = time.time()
            new_assembly_Prim_x(random_pile, score_matrix, gap_cost)
            end_func_time = time.time()
            time_taken_Prim = end_func_time - start_func_time

        for random_pile in random_sequences_collection:
            start_func_time = time.time()
            new_assembly_Gus_x(random_pile, score_matrix, gap_cost)
            end_func_time = time.time()
            time_taken_Gus = end_func_time - start_func_time

    elif mode == 'family':
        strings_in_families = make_strings_in_families(nr_of_fams, str_pr_fam, len_str, internal_var, var_between_fams)
        start_func_time = time.time()
        new_assembly_gradual_x(strings_in_families, score_matrix, gap_cost)
        end_func_time = time.time()
        time_taken_Krus = end_func_time - start_func_time

        start_func_time = time.time()
        new_assembly_Prim_x(strings_in_families, score_matrix, gap_cost)
        end_func_time = time.time()
        time_taken_Prim = end_func_time - start_func_time

        start_func_time = time.time()
        new_assembly_Gus_x(strings_in_families, score_matrix, gap_cost)
        end_func_time = time.time()
        time_taken_Gus = end_func_time - start_func_time

    else:
        print("That's not a valid mode, try 'family' or 'random'!")

    
    if mode == 'random':
        return nr_of_str, len_str, mode, time_taken_Krus, time_taken_Prim, time_taken_Gus
    if mode == 'family':
        return len_str, nr_of_fams, str_pr_fam, internal_var, var_between_fams, mode, time_taken_Krus, time_taken_Prim, time_taken_Gus

# Example usage
(test_many_parameters_no_internal_reps_time(len_str=15, nr_of_fams=3, str_pr_fam=3, internal_var=0.1, var_between_fams=0.9, mode='family'))


(15,
 3,
 3,
 0.1,
 0.9,
 'family',
 0.18258023262023926,
 0.21428442001342773,
 0.1840071678161621)

In [5]:
import time
import random


def test_many_parameters_no_internal_reps_time_new_prim(nr_of_str=10, len_str=20, nr_of_fams=2, str_pr_fam=5, internal_var=0.1, var_between_fams=0.9, mode='random'):
    time_taken_Krus = float('inf')
    time_taken_Prim = float('inf')
    time_taken_Gus = float('inf')

    start_time = time.time()

    if mode == 'random':
        random_sequences_collection = []
        random_pile = []
        for j in range(0, nr_of_str):
            a_sequence = []
            for k in range(0, len_str):
                a_sequence.append(random.choice(['a', 'c', 't', 'g']))
            a_sequence = ''.join(a_sequence)
            random_pile.append(a_sequence)
        random_sequences_collection.append(random_pile)
        for random_pile in random_sequences_collection:
            start_func_time = time.time()
            new_assembly_gradual_x(random_pile, score_matrix, gap_cost)
            end_func_time = time.time()
            time_taken_Krus = end_func_time - start_func_time

        for random_pile in random_sequences_collection:
            start_func_time = time.time()
            new_assembly_Primzeyy_x(random_pile, score_matrix, gap_cost)
            end_func_time = time.time()
            time_taken_Prim = end_func_time - start_func_time

        for random_pile in random_sequences_collection:
            start_func_time = time.time()
            new_assembly_Gus_x(random_pile, score_matrix, gap_cost)
            end_func_time = time.time()
            time_taken_Gus = end_func_time - start_func_time

    elif mode == 'family':
        strings_in_families = make_strings_in_families(nr_of_fams, str_pr_fam, len_str, internal_var, var_between_fams)
        start_func_time = time.time()
        new_assembly_gradual_x(strings_in_families, score_matrix, gap_cost)
        end_func_time = time.time()
        time_taken_Krus = end_func_time - start_func_time

        start_func_time = time.time()
        new_assembly_Primzeyy_x(strings_in_families, score_matrix, gap_cost)
        end_func_time = time.time()
        time_taken_Prim = end_func_time - start_func_time

        start_func_time = time.time()
        new_assembly_Gus_x(strings_in_families, score_matrix, gap_cost)
        end_func_time = time.time()
        time_taken_Gus = end_func_time - start_func_time

    else:
        print("That's not a valid mode, try 'family' or 'random'!")

    
    if mode == 'random':
        return nr_of_str, len_str, mode, time_taken_Krus, time_taken_Prim, time_taken_Gus
    if mode == 'family':
        return len_str, nr_of_fams, str_pr_fam, internal_var, var_between_fams, mode, time_taken_Krus, time_taken_Prim, time_taken_Gus

# Example usage
(test_many_parameters_no_internal_reps_time(len_str=15, nr_of_fams=3, str_pr_fam=3, internal_var=0.1, var_between_fams=0.9, mode='family'))


(15,
 3,
 3,
 0.1,
 0.9,
 'family',
 0.15789318084716797,
 0.12912201881408691,
 0.10211181640625)

In [12]:
from IPython.display import clear_output
#for random time
repeats=3
mode='random'
str_len=[10,50,100,200,300,500,1000]
nr_str=[5,10,15]

csv_file_path="testing_outputs/1_random_strs_tests_time_new_prim.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile, delimiter=',', quotechar=' ', quoting=csv.QUOTE_NONE)
                            csvwriter.writerow(["nr_of_str","str_len","mode","Kruskal","Prim","Gusfield"])

for i in range(0,repeats):
    for lens in str_len:
        for nr in nr_str:

            nr_of_str,len_str,mode,time_Krus,time_Prim,time_Gus=test_many_parameters_no_internal_reps_time_new_prim(nr,lens,mode='random')
            with open(csv_file_path, 'a', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow([nr_of_str,len_str,mode,time_Krus,time_Prim,time_Gus])
            clear_output(wait=True)











































I WENT PAST MAKING THE EDGES
just gonna print the new min span edges
[array(['', '2032', '0', '8'], dtype='<U21'), array(['', '2050', '0', '10'], dtype='<U21'), array(['', '2044', '7', '10'], dtype='<U21'), array(['', '2039', '3', '7'], dtype='<U21'), array(['', '2030', '3', '13'], dtype='<U21'), array(['', '2040', '1', '13'], dtype='<U21'), array(['', '2043', '7', '12'], dtype='<U21'), array(['', '2046', '7', '9'], dtype='<U21'), array(['', '2049', '2', '10'], dtype='<U21'), array(['', '2061', '2', '14'], dtype='<U21'), array(['', '2064', '8', '11'], dtype='<U21'), array(['', '2050', '5', '11'], dtype='<U21'), array(['', '2065', '6', '13'], dtype='<U21'), array(['', '2067', '3', '4'], dtype='<U21')]
['' '2032' '0' '8']
2032
ILL JUST PRINT MIN SPAN EGEDS
[array(['', '2032', '0', '8'], dtype='<U21'), array(['', '2050', '0', '10'], dtype='<U21'), array(['', '2044', '7', '10'], dtype='<U21'), array(['', '2039', '3', '7'], dtype='<U21'), array(['',

In [14]:
#for familiy time

repeats=3
mode='family'
nr_fams=[2]
str_pr_fam=[10]
internal_var=[0.1,0.9]
var_between_fams=[0.1,0.9]
str_len=[50,100,200,300]
for x in var_between_fams:
    print(x)
csv_file_path="testing_outputs/1_familiy_outputs_testing_time_new_prim.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow(["str_len","nr_of_fams","str_pr_fam","internal_var","external_var","mode","Kruskal","Prim","Gusfield"])

for i in range(0,repeats):
    for fam_num in nr_fams:
        for str_pr_fam_nr in str_pr_fam:
            for int_var in internal_var:
                for ex_var in var_between_fams:
                    for str_l in str_len:

                        len_str,nr_of_fams,strs_pr_fam,inte_var,var_between_fam,mode,time_Krus,time_Prim,time_Gus=test_many_parameters_no_internal_reps_time_new_prim(len_str=str_l,nr_of_fams=fam_num,str_pr_fam=str_pr_fam_nr,internal_var=int_var,var_between_fams=ex_var,mode=mode)
                        with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow([len_str,nr_of_fams,strs_pr_fam,inte_var,var_between_fam,mode,time_Krus,time_Prim,time_Gus])
                    clear_output(wait=True)


























































I WENT PAST MAKING THE EDGES
just gonna print the new min span edges
[array(['', '95', '0', '19'], dtype='<U21'), array(['', '89', '13', '19'], dtype='<U21'), array(['', '91', '12', '19'], dtype='<U21'), array(['', '97', '0', '16'], dtype='<U21'), array(['', '86', '7', '16'], dtype='<U21'), array(['', '97', '10', '16'], dtype='<U21'), array(['', '77', '10', '14'], dtype='<U21'), array(['', '94', '10', '17'], dtype='<U21'), array(['', '95', '10', '15'], dtype='<U21'), array(['', '93', '11', '15'], dtype='<U21'), array(['', '98', '0', '4'], dtype='<U21'), array(['', '98', '6', '14'], dtype='<U21'), array(['', '99', '0', '1'], dtype='<U21'), array(['', '100', '0', '3'], dtype='<U21'), array(['', '100', '2', '11'], dtype='<U21'), array(['', '100', '8', '17'], dtype='<U21'), array(['', '103', '5', '11'], dtype='<U21'), array(['', '106', '1', '9'], dtype='<U21'), array(['', '108', '12', '18'], dtype='<U21')]
['' '95' '0' '19']
95
ILL J

In [11]:
#for families cost but with very different internal var to check the "actual" critical values
repeats=1
mode='family'
nr_fams=[2]
str_pr_fam=[10]
internal_var=np.arange(0.41, 0.6, 0.01).tolist()
var_between_fams=[1]
str_len=[50]
for x in var_between_fams:
    print(x)
csv_file_path="testing_outputs/families_diff_internal_var2.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow(["str_len","nr_of_fams","str_pr_fam","internal_var","external_var","mode","Kruskal","Prim","Gusfield"])

for i in range(0,repeats):
    for fam_num in nr_fams:
        for str_pr_fam_nr in str_pr_fam:
            for int_var in internal_var:
                for ex_var in var_between_fams:
                    for str_l in str_len:

                        len_str,nr_of_fams,strs_pr_fam,inte_var,var_between_fam,mode,costs_Krus,costs_Prim,costs_Gus=test_many_parameters_no_internal_reps(len_str=str_l,nr_of_fams=fam_num,internal_var=int_var,str_pr_fam=str_pr_fam_nr,var_between_fams=ex_var,mode=mode)

                        with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow([len_str,nr_of_fams,strs_pr_fam,inte_var,var_between_fam,mode,costs_Krus,costs_Prim,costs_Gus])

1


In [5]:
np.arange(0, 1, 0.01).tolist()

[0.0,
 0.01,
 0.02,
 0.03,
 0.04,
 0.05,
 0.06,
 0.07,
 0.08,
 0.09,
 0.1,
 0.11,
 0.12,
 0.13,
 0.14,
 0.15,
 0.16,
 0.17,
 0.18,
 0.19,
 0.2,
 0.21,
 0.22,
 0.23,
 0.24,
 0.25,
 0.26,
 0.27,
 0.28,
 0.29,
 0.3,
 0.31,
 0.32,
 0.33,
 0.34,
 0.35000000000000003,
 0.36,
 0.37,
 0.38,
 0.39,
 0.4,
 0.41000000000000003,
 0.42,
 0.43,
 0.44,
 0.45,
 0.46,
 0.47000000000000003,
 0.48,
 0.49,
 0.5,
 0.51,
 0.52,
 0.53,
 0.54,
 0.55,
 0.56,
 0.5700000000000001,
 0.58,
 0.59,
 0.6,
 0.61,
 0.62,
 0.63,
 0.64,
 0.65,
 0.66,
 0.67,
 0.68,
 0.6900000000000001,
 0.7000000000000001,
 0.71,
 0.72,
 0.73,
 0.74,
 0.75,
 0.76,
 0.77,
 0.78,
 0.79,
 0.8,
 0.81,
 0.8200000000000001,
 0.8300000000000001,
 0.84,
 0.85,
 0.86,
 0.87,
 0.88,
 0.89,
 0.9,
 0.91,
 0.92,
 0.93,
 0.9400000000000001,
 0.9500000000000001,
 0.96,
 0.97,
 0.98,
 0.99]